# Yahtzee

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras as tfk
from tensorflow.keras.callbacks import TensorBoard

%matplotlib inline

__MODEL_PATH = 'models'
__TENSOR_LOG_DIR = 'logs'

## Dataset

Let's start with looking at the provided dataset:

In [2]:
df = pd.read_csv('yahtzee-dataset.csv')
print('Labels:', df.label.unique())

df.head(10)

Labels: ['nothing' 'small-straight' 'three-of-a-kind' 'large-straight'
 'full-house' 'four-of-a-kind' 'yathzee']


,dice1,dice2,dice3,dice4,dice5,label
0,3,6,6,2,5,nothing
1,3,6,1,3,4,nothing
2,2,2,5,5,3,nothing
3,1,3,6,6,1,nothing
4,1,4,6,3,5,small-straight
5,4,1,4,3,1,nothing
6,4,4,4,6,2,three-of-a-kind
7,3,2,5,6,3,nothing
8,3,4,3,6,2,nothing
9,3,3,1,5,4,nothing


In order to classify these categorical labels, we have to 'one-hot encode' them:

In [3]:
one_hot_df = pd.get_dummies(df, prefix=['label'])
one_hot_df.head(10)

,dice1,dice2,dice3,dice4,dice5,label_four-of-a-kind,label_full-house,label_large-straight,label_nothing,label_small-straight,label_three-of-a-kind,label_yathzee
0,3,6,6,2,5,0,0,0,1,0,0,0
1,3,6,1,3,4,0,0,0,1,0,0,0
2,2,2,5,5,3,0,0,0,1,0,0,0
3,1,3,6,6,1,0,0,0,1,0,0,0
4,1,4,6,3,5,0,0,0,0,1,0,0
5,4,1,4,3,1,0,0,0,1,0,0,0
6,4,4,4,6,2,0,0,0,0,0,1,0
7,3,2,5,6,3,0,0,0,1,0,0,0
8,3,4,3,6,2,0,0,0,1,0,0,0
9,3,3,1,5,4,0,0,0,1,0,0,0


Before we can train any model, we have to split the data and the labels into X and Y:

In [4]:
X = one_hot_df.iloc[:,:5].copy()
Y = one_hot_df.iloc[:,5:].copy()

X.head(5)

,dice1,dice2,dice3,dice4,dice5
0,3,6,6,2,5
1,3,6,1,3,4
2,2,2,5,5,3
3,1,3,6,6,1
4,1,4,6,3,5


We also split the dataset into a 9:1 split for training and validating the model:

In [5]:
split = int(len(X.index) * .9)
X_train = X.iloc[:split]
X_valid = X.iloc[split:]
Y_train = Y.iloc[:split]
Y_valid = Y.iloc[split:]

print('Split X:', X_train.shape, X_valid.shape)
print('Split Y:', Y_train.shape, Y_valid.shape)

Y_train.head(5)

Split X: (5248, 5) (584, 5)
Split Y: (5248, 7) (584, 7)


,label_four-of-a-kind,label_full-house,label_large-straight,label_nothing,label_small-straight,label_three-of-a-kind,label_yathzee
0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0
2,0,0,0,1,0,0,0
3,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0


## Model

We designed several models:

In [6]:
def model_1(shape):
    """
    Single hidden layer with 128 neurons ans Sigmoid activation function.
    """
    return tfk.models.Sequential([
        tfk.layers.Dense(128,      activation='sigmoid', input_shape=(shape[0],), name='l1'),
        tfk.layers.Dense(shape[1], activation='softmax',                          name='output')
    ])

We can choose a model to train and test:

In [7]:
model_fn = model_1
model = model_fn((X.shape[1], Y.shape[1]))

We configure TensorBoard to get a good look on the performance of our model:

In [8]:
model.summary()
tensor_board = TensorBoard(log_dir=os.path.join(__TENSOR_LOG_DIR, model_fn.__name__), histogram_freq=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
l1 (Dense)                   (None, 128)               768       
_________________________________________________________________
output (Dense)               (None, 7)                 903       
Total params: 1,671
Trainable params: 1,671
Non-trainable params: 0
_________________________________________________________________


We choose an optimizer, a loss functon and metrics:

In [9]:
# Compile the model and use the Adam optimizer, Cross Entropy loss function and accuracy metric
optimizer = tfk.optimizers.Adam(lr=.001, decay=.0)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

We train the model using a certain batch size and for a number of epochs:

In [10]:
epochs = 10
batch_size = 100
validation_split = .1

# Train the model
model.fit(x=X_train, 
          y=Y_train,
          epochs=epochs,
          batch_size=batch_size,
          validation_split=validation_split,
          verbose=1,
          shuffle=True,
          callbacks=[tensor_board])

ValueError: Error when checking target: expected output to have shape (1,) but got array with shape (7,)

We validate the model with the data it has not seen yet:

In [ ]:
# Validate the model with unseen data
score = model.evaluate(X_valid, Y_valid)

# Print test accuracy
print('\n', 'Test accuracy:', score[1])

We save the model that worked best:

In [ ]:
# Save the model
model.save(os.path.join(__MODEL_PATH, model_fn.__name__)